# Load data

### Imports

In [1]:
import os
import pandas as pd
from sklearn import preprocessing
import numpy as np

### P-Value 0.01

In [2]:
files = os.listdir('C:/Users/Enaut/PycharmProjects/initialTestingProject/resultsCsv/mergedCsv/')
df_total_001 = pd.DataFrame(columns=["Origin_program", "Operator", "Gate", "Position", "Killed"])

for file in files:
    df_new = pd.read_csv('C:/Users/Enaut/PycharmProjects/initialTestingProject/resultsCsv/mergedCsv/' + str(file))
    del df_new['Qubits']
    df_total_001 = pd.concat([df_total_001, df_new], ignore_index=True)
    df_new = df_new.iloc[0:0]

    
df_total_001.info()
df_total_001.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723079 entries, 0 to 723078
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Origin_program  723079 non-null  object
 1   Operator        723079 non-null  object
 2   Gate            723079 non-null  object
 3   Position        723079 non-null  object
 4   Killed          723079 non-null  object
dtypes: object(5)
memory usage: 27.6+ MB


,Origin_program,Operator,Gate,Position,Killed
0,ae_indep_qiskit_2,Add,cx,1,WOO
1,ae_indep_qiskit_2,Add,cx,2,WOO
2,ae_indep_qiskit_2,Add,cx,3,Survivor
3,ae_indep_qiskit_2,Add,cx,4,WOO
4,ae_indep_qiskit_2,Add,cx,5,Survivor


### Recalculate position

In [3]:
df_mapping = pd.read_csv('position_mapping.csv')

df_total_001['Line'] = df_total_001['Position']
x = 0
y = 0
for index, row in df_total_001.iterrows():
    df_total_001.loc[index, 'Line'] = int(df_mapping[(df_mapping['program'] == df_total_001.loc[index, 'Origin_program']) & (df_mapping['position'] == df_total_001.loc[index, 'Position'])]['line'])
    x = x + 1
    if x > 10000:
        y = y + 10000
        x = 0
        print(y)


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000


In [4]:
df_total_001

,Origin_program,Operator,Gate,Position,Killed,Line
0,ae_indep_qiskit_2,Add,cx,1,WOO,6
1,ae_indep_qiskit_2,Add,cx,2,WOO,8
2,ae_indep_qiskit_2,Add,cx,3,Survivor,9
3,ae_indep_qiskit_2,Add,cx,4,WOO,11
4,ae_indep_qiskit_2,Add,cx,5,Survivor,12
...,...,...,...,...,...,...
723074,wstate_indep_qiskit_27,Replace,z,8,WOO,11
723075,wstate_indep_qiskit_27,Replace,z,91,WOO,94
723076,wstate_indep_qiskit_27,Replace,z,93,WOO,96
723077,wstate_indep_qiskit_27,Replace,z,95,WOO,98


### New column for replace operator

In [6]:
AllGates = ("x", "h", "p", "t", "s", "z", "y", "id", "rx", "ry", "rz", "sx", "swap", "rzz", "rxx", "cx", "cz", "ccx", "cswap")
def getGateForPosition(origin_name, position):
    splitChar = 92
    folder_path = r'C:\Users\Enaut\Desktop\Enaut\PhD\Experiments\Final_experiments_ICSE\SelectedPrograms\Qiskit\Origin_Programs'
    origin_path = folder_path + chr(splitChar) + origin_name + '_rewriteCircuit.py'
    x = 1
    f = open(origin_path)
    line = f.readline()
    while x <= position:
        if ('qc' in line) and ("QuantumCircuit" not in line):
            temp = line.split(".", 1)
            temp2 = temp[1].split("(")
            if temp2[0] in AllGates:
                x = x + 1
                gate = temp2[0]
        line = f.readline()

    return gate

In [7]:
df_total_001['New_gate'] = ""
for index, row in df_total_001.iterrows():
    if row['Operator'] == 'Replace':
        df_total_001.loc[index, 'New_gate'] = row['Gate']
        df_total_001.loc[index, 'Gate'] = getGateForPosition(row['Origin_program'], row['Position'])
    else:
        df_total_001.loc[index, 'New_gate'] = row['Gate']
df_total_001

,Origin_program,Operator,Gate,Position,Killed,Line,New_gate
0,ae_indep_qiskit_2,Add,cx,1,WOO,6,cx
1,ae_indep_qiskit_2,Add,cx,2,WOO,8,cx
2,ae_indep_qiskit_2,Add,cx,3,Survivor,9,cx
3,ae_indep_qiskit_2,Add,cx,4,WOO,11,cx
4,ae_indep_qiskit_2,Add,cx,5,Survivor,12,cx
...,...,...,...,...,...,...,...
723074,wstate_indep_qiskit_27,Replace,ry,8,WOO,11,z
723075,wstate_indep_qiskit_27,Replace,ry,91,WOO,94,z
723076,wstate_indep_qiskit_27,Replace,ry,93,WOO,96,z
723077,wstate_indep_qiskit_27,Replace,ry,95,WOO,98,z


### Program characteristics

In [8]:
df_characteristics = pd.read_excel('C:/Users/Enaut/Desktop/Enaut/PhD/Experiments/Final_experiments_ICSE/SelectedPrograms/programs_characteristics.xlsx', sheet_name='characteristics')

In [9]:
df_characteristics.head()

,Unnamed: 0,group,qubits,gates,measurement_gates,depth,singlequbit_gates,multiqubit_gates,entangled_qubits
0,ae_indep_qiskit_2,aeindepqiskit,2,8,2,6,5,3,0
1,ae_indep_qiskit_3,aeindepqiskit,3,15,3,12,9,6,0
2,ae_indep_qiskit_4,aeindepqiskit,4,23,4,18,13,10,0
3,ae_indep_qiskit_5,aeindepqiskit,5,32,5,24,17,15,0
4,ae_indep_qiskit_6,aeindepqiskit,6,42,6,30,21,21,0


In [10]:
df_characteristics.rename(columns = {'Unnamed: 0':'Origin_program'}, inplace = True)
df_characteristics.rename(columns = {'group':'algorithm'}, inplace = True)

### Merge info

In [11]:
complete_001 = pd.merge(df_characteristics, df_total_001, on="Origin_program")
complete_001.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 723079 entries, 0 to 723078
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Origin_program     723079 non-null  object
 1   algorithm          723079 non-null  object
 2   qubits             723079 non-null  int64 
 3   gates              723079 non-null  int64 
 4   measurement_gates  723079 non-null  int64 
 5   depth              723079 non-null  int64 
 6   singlequbit_gates  723079 non-null  int64 
 7   multiqubit_gates   723079 non-null  int64 
 8   entangled_qubits   723079 non-null  int64 
 9   Operator           723079 non-null  object
 10  Gate               723079 non-null  object
 11  Position           723079 non-null  object
 12  Killed             723079 non-null  object
 13  Line               723079 non-null  object
 14  New_gate           723079 non-null  object
dtypes: int64(7), object(8)
memory usage: 88.3+ MB


In [12]:
#Clean algorithm name
complete_001['algorithm'] = complete_001['algorithm'].str.replace('indepqiskit','')

In [13]:
complete_001['algorithm'].unique()

array(['ae', 'dj', 'ghz', 'graphstate', 'groundstatelarge',
       'groundstatemedium', 'groundstatesmall', 'grover-noancilla',
       'grover-v-chain', 'portfolioqaoa', 'portfoliovqe', 'pricingcall',
       'pricingput', 'qaoa', 'qft', 'qftentangled', 'qgan', 'qpeexact',
       'qpeinexact', 'qwalk-noancilla', 'qwalk-v-chain', 'realamprandom',
       'routing', 'su2random', 'tsp', 'twolocalrandom', 'vqe', 'wstate'],
      dtype=object)

In [14]:
#Create algorithm groups into base algorithms
complete_001.loc[(complete_001.algorithm == 'dj'), 'algorithm_group'] = 'dj'
complete_001.loc[(complete_001.algorithm == 'ghz'), 'algorithm_group'] = 'ghz'
complete_001.loc[(complete_001.algorithm == 'graphstate'), 'algorithm_group'] = 'graphstate'
complete_001.loc[(complete_001.algorithm == 'groundstatelarge') | (complete_001.algorithm == 'groundstatemedium') | (complete_001.algorithm == 'groundstatesmall') | (complete_001.algorithm == 'portfoliovqe') | (complete_001.algorithm == 'vqe') | (complete_001.algorithm == 'su2random') | (complete_001.algorithm == 'tsp') | (complete_001.algorithm == 'realamprandom') | (complete_001.algorithm == 'twolocalrandom') | (complete_001.algorithm == 'routing'), 'algorithm_group'] = 'vqe'
complete_001.loc[(complete_001.algorithm == 'grover-noancilla') | (complete_001.algorithm == 'grover-v-chain'), 'algorithm_group'] = 'grover'
complete_001.loc[(complete_001.algorithm == 'portfolioqaoa') | (complete_001.algorithm == 'qaoa'), 'algorithm_group'] = 'qaoa'
complete_001.loc[(complete_001.algorithm == 'pricingcall') | (complete_001.algorithm == 'pricingput') | (complete_001.algorithm == 'ae'), 'algorithm_group'] = 'ae'
complete_001.loc[(complete_001.algorithm == 'qft') | (complete_001.algorithm == 'qftentangled'), 'algorithm_group'] = 'qft'
complete_001.loc[(complete_001.algorithm == 'qgan'), 'algorithm_group'] = 'qgan'
complete_001.loc[(complete_001.algorithm == 'qpeexact') | (complete_001.algorithm == 'qpeinexact'), 'algorithm_group'] = 'qpe'
complete_001.loc[(complete_001.algorithm == 'qwalk-noancilla') | (complete_001.algorithm == 'qwalk-v-chain'), 'algorithm_group'] = 'qwalk'
complete_001.loc[(complete_001.algorithm == 'wstate'), 'algorithm_group'] = 'wstate'


In [15]:
#Classify algorithm depending on the scalability on mqtbench
complete_001['dominant_state']='no'
dominant_groups = ('ae', 'vqe', 'grover','qaoa','qpe')
complete_001.loc[complete_001['algorithm_group'].isin(dominant_groups), 'dominant_state'] = 'yes'



In [16]:
complete_001['algorithm_group'].unique()

array(['ae', 'dj', 'ghz', 'graphstate', 'vqe', 'grover', 'qaoa', 'qft',
       'qgan', 'qpe', 'qwalk', 'wstate'], dtype=object)

In [17]:
#Postions with percent
complete_001["Position_percent"] = (100*complete_001['Line'].astype(int))/(complete_001['qubits'].astype(int) + complete_001['gates'].astype(int))


In [18]:
#Convert positions to integer
complete_001["Position_percent"] = complete_001["Position_percent"].astype(int)


In [19]:
#Positions by 10%
complete_001.loc[complete_001.Position_percent <= 10, 'Position_by_10'] = 10
complete_001.loc[(complete_001.Position_percent > 10) & (complete_001.Position_percent <= 20), 'Position_by_10'] = 20
complete_001.loc[(complete_001.Position_percent > 20) & (complete_001.Position_percent <= 30), 'Position_by_10'] = 30
complete_001.loc[(complete_001.Position_percent > 30) & (complete_001.Position_percent <= 40), 'Position_by_10'] = 40
complete_001.loc[(complete_001.Position_percent > 40) & (complete_001.Position_percent <= 50), 'Position_by_10'] = 50
complete_001.loc[(complete_001.Position_percent > 50) & (complete_001.Position_percent <= 60), 'Position_by_10'] = 60
complete_001.loc[(complete_001.Position_percent > 60) & (complete_001.Position_percent <= 70), 'Position_by_10'] = 70
complete_001.loc[(complete_001.Position_percent > 70) & (complete_001.Position_percent <= 80), 'Position_by_10'] = 80
complete_001.loc[(complete_001.Position_percent > 80) & (complete_001.Position_percent <= 90), 'Position_by_10'] = 90
complete_001.loc[(complete_001.Position_percent > 90), 'Position_by_10'] = 100


In [20]:
#Combination of operator with gate
complete_001["Operator_gate"] = complete_001['Operator'].astype(str) +"_"+ complete_001["Gate"].astype(str)


In [21]:
#Combination with operator position
complete_001["Operator_position"] = complete_001['Operator'].astype(str) +"_"+ complete_001["Position_by_10"].astype(str)


In [22]:
#Combination gate and position
complete_001["Gate_position"] = complete_001['Gate'].astype(str) +"_"+ complete_001["Position_by_10"].astype(str)


In [23]:
#Combination operator, gate adn position
complete_001["Operator_gate_position"] = complete_001['Operator'].astype(str) +"_"+ complete_001["Gate"].astype(str) +"_"+ complete_001["Position_by_10"].astype(str)


In [24]:
#Gate group by number of qubits
SingleQubit = ("x", "h", "p", "t", "s", "z", "y", "id", "rx", "ry", "rz", "sx", "u", "u1", "u2", "u3")
MultiQubit = ("swap", "rzz", "rxx", "cx", "cz", "ccx", "cswap")
complete_001.loc[complete_001['Gate'].isin(SingleQubit), 'Gate_size'] = 'Single'
complete_001.loc[complete_001['Gate'].isin(MultiQubit), 'Gate_size'] = 'Multi'


In [25]:
#Gate group by type

T = ("t",)
Phase = ("p","s",)
Pauli = ("x", "id", "z", "y", "sx")
Rotation = ("rx", "ry", "rz", "rzz", "rxx")
Hadamard = ("h",)
Controlledgates = ("cx", "cz", "ccx", "cswap")
SwapGate = ("swap",)

complete_001.loc[complete_001['Gate'].isin(T), 'Gate_type'] = 'T'
complete_001.loc[complete_001['Gate'].isin(Phase), 'Gate_type'] = 'Phase'
complete_001.loc[complete_001['Gate'].isin(Pauli), 'Gate_type'] = 'Pauli'
complete_001.loc[complete_001['Gate'].isin(Rotation), 'Gate_type'] = 'Rotation'
complete_001.loc[complete_001['Gate'].isin(Hadamard), 'Gate_type'] = 'Hadamard'
complete_001.loc[complete_001['Gate'].isin(Controlledgates), 'Gate_type'] = 'Controlled'
complete_001.loc[complete_001['Gate'].isin(SwapGate), 'Gate_type'] = 'SwapGate'



In [26]:
complete_001

,Origin_program,algorithm,qubits,gates,measurement_gates,depth,singlequbit_gates,multiqubit_gates,entangled_qubits,Operator,...,algorithm_group,dominant_state,Position_percent,Position_by_10,Operator_gate,Operator_position,Gate_position,Operator_gate_position,Gate_size,Gate_type
0,ae_indep_qiskit_2,ae,2,8,2,6,5,3,0,Add,...,ae,yes,60,60.0,Add_cx,Add_60.0,cx_60.0,Add_cx_60.0,Multi,Controlled
1,ae_indep_qiskit_2,ae,2,8,2,6,5,3,0,Add,...,ae,yes,80,80.0,Add_cx,Add_80.0,cx_80.0,Add_cx_80.0,Multi,Controlled
2,ae_indep_qiskit_2,ae,2,8,2,6,5,3,0,Add,...,ae,yes,90,90.0,Add_cx,Add_90.0,cx_90.0,Add_cx_90.0,Multi,Controlled
3,ae_indep_qiskit_2,ae,2,8,2,6,5,3,0,Add,...,ae,yes,110,100.0,Add_cx,Add_100.0,cx_100.0,Add_cx_100.0,Multi,Controlled
4,ae_indep_qiskit_2,ae,2,8,2,6,5,3,0,Add,...,ae,yes,120,100.0,Add_cx,Add_100.0,cx_100.0,Add_cx_100.0,Multi,Controlled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723074,wstate_indep_qiskit_27,wstate,27,106,27,55,53,53,0,Replace,...,wstate,no,8,10.0,Replace_ry,Replace_10.0,ry_10.0,Replace_ry_10.0,Single,Rotation
723075,wstate_indep_qiskit_27,wstate,27,106,27,55,53,53,0,Replace,...,wstate,no,70,70.0,Replace_ry,Replace_70.0,ry_70.0,Replace_ry_70.0,Single,Rotation
723076,wstate_indep_qiskit_27,wstate,27,106,27,55,53,53,0,Replace,...,wstate,no,72,80.0,Replace_ry,Replace_80.0,ry_80.0,Replace_ry_80.0,Single,Rotation
723077,wstate_indep_qiskit_27,wstate,27,106,27,55,53,53,0,Replace,...,wstate,no,73,80.0,Replace_ry,Replace_80.0,ry_80.0,Replace_ry_80.0,Single,Rotation


In [27]:
complete_001.to_csv("merged_data_001.csv",index=False)